In [23]:
import pandas as pd
import ipaddress
import dns.resolver
import dns.reversename
import pygeoip
import pandas as pd
import matplotlib.pyplot as plt
    
#data 
data_normal = pd.read_parquet('data0.parquet')
data_attack = pd.read_parquet('test0.parquet')

# Get the organization name for an IP address
geo1=pygeoip.GeoIP('./GeoIP_DBs/GeoIP.dat')
geo2=pygeoip.GeoIP('./GeoIP_DBs/GeoIPASNum.dat')

def get_countryname(ip):
    return geo1.country_name_by_addr(ip)

# Verify if all ports are equal in data_normal vs data_attack
if (data_normal['port'].unique() != data_attack['port'].unique()).all():
    print("Different ports in data_normal vs data_attack")

# Verify if all protocols are equal in data_normal vs data_attack
if (data_normal['proto'].unique() != data_attack['proto'].unique()).all():
    print("Different protocols in data_normal vs data_attack")

# Put a label for each dst_ip with the country name
data_normal['dst_country'] = data_normal['dst_ip'].apply(get_countryname)
data_attack['dst_country'] = data_attack['dst_ip'].apply(get_countryname)

# Show the countries that not exists in data_normal
print("Countries that not exists in data_normal:")




for country in data_attack['dst_country'].unique():
    if country not in data_normal['dst_country'].unique() and len(data_attack.loc[data_attack['dst_country']==country]) > 100:
        print(country + "with " + str(len(data_attack.loc[data_attack['dst_country']==country])) + " flows")



Countries that not exists in data_normal:
Russian Federationwith 1191 flows


In [3]:
# Check possilbe dangerous src_ip in data_attack

# Get the frequency of each src_ip in data_normal
data_normal_src_ip = data_normal.groupby('src_ip')['src_ip'].count()
data_normal_src_ip = data_normal_src_ip.sort_values(ascending=False)

# Get the frequency of each src_ip in data_attack
data_attack_src_ip = data_attack.groupby('src_ip')['src_ip'].count()
data_attack_src_ip = data_attack_src_ip.sort_values(ascending=False)

# Get the src_ip that is generating more traffic than 99% of the src_ip in data_normal
data_attack_src_ip = data_attack_src_ip[data_attack_src_ip > data_normal_src_ip.quantile(0.99)]
data_attack_src_ip = data_attack_src_ip.sort_values(ascending=False)

print("Ips that are generating more traffic than 99% othersl:")
for ip in data_attack_src_ip.index:
    print("IP: " + ip + " - " + str(data_attack_src_ip[ip]) + " packets")

Ips that are generating more traffic than 99% othersl:
IP: 192.168.100.176 - 79884 packets
IP: 192.168.100.188 - 45942 packets
IP: 192.168.100.49 - 12103 packets
IP: 192.168.100.75 - 10846 packets


In [4]:
# Assuming 'up_bytes' and 'down_bytes' are the columns representing the bytes in the dataset

#  Sum 'up_bytes' and 'down_bytes'
data_normal['bytes'] = data_normal['up_bytes'] + data_normal['down_bytes']
data_attack['bytes'] = data_attack['up_bytes'] + data_attack['down_bytes']

# Sum all bytes by dst_country
data_normal_bytes = data_normal.groupby('dst_country')['bytes'].sum()
data_attack_bytes = data_attack.groupby('dst_country')['bytes'].sum()

# verify the difference between data_normal_bytes and data_attack_bytes
data_diff_bytes = data_attack_bytes - data_normal_bytes
data_diff_bytes = data_diff_bytes.sort_values(ascending=False)
data_diff_bytes = data_diff_bytes.dropna()

# Calculate the percentage of difference
data_diff_bytes = data_diff_bytes / data_normal_bytes * 100

# Show the countries that have a difference of more than 100% in data_attack
print("Countries that have a difference of more than 50% in data_attack:")
for country in data_diff_bytes.index:
    if data_diff_bytes[country] > 50:
        # check if i need '\t'
        # i want that the second column start in the same place
        if len(country) < 8:
            print(country + "\t\t\t" + str(int(data_diff_bytes[country])) + "%")
        elif len(country) < 16:
            print(country + "\t\t" + str(int(data_diff_bytes[country])) + "%")
        else:
            print(country + "\t" + str(int(data_diff_bytes[country])) + "%")



Countries that have a difference of more than 50% in data_attack:
Chile			204%
China			529%
Indonesia		99%
Israel			64%
Korea, Republic of	61%


In [32]:
# check for connections between our private ips and Chile, China, Indonesia, Israel and Korea, Republic of

# new connections to chile
chile_normal = data_normal[data_normal['dst_country'].str.contains('chile', case=False)]
chile_attack = data_attack[data_attack['dst_country'].str.contains('chile', case=False)]
merged = pd.merge(chile_normal, chile_attack, on='dst_ip', how='outer', indicator=True)
dif = merged[merged["_merge"] != 'right_only']
print("New Chile IPs")
for ip in dif['dst_ip'].unique():
    print("IP "+ str(ip) +" flows : "+str(len(dif.loc[dif['dst_ip']==ip])))

# new connections to china
china_normal = data_normal[data_normal['dst_country'].str.contains('china', case=False)]
china_attack = data_attack[data_attack['dst_country'].str.contains('china', case=False)]
merged = pd.merge(china_normal, china_attack, on='dst_ip', how='outer', indicator=True)
dif = merged[merged["_merge"] != 'right_only']
print("New China IPs")
for ip in dif['dst_ip'].unique():
    print(str(ip) +" -> "+str(len(dif.loc[dif['dst_ip']==ip])))

# no need to check others

New Chile IPs
IP 40.97.14.201 flows : 7
IP 40.97.14.161 flows : 5
New China IPs
15.230.41.82 -> 2
13.34.7.70 -> 8
150.222.135.245 -> 1
13.34.7.87 -> 9
15.230.41.191 -> 3
15.230.74.35 -> 2
150.222.135.193 -> 2
15.230.73.206 -> 1
150.222.135.49 -> 10
15.230.73.254 -> 11
150.222.135.120 -> 2
150.222.135.26 -> 1
13.34.7.125 -> 6
13.34.7.97 -> 1
150.222.135.249 -> 1
15.230.141.187 -> 3
15.230.141.38 -> 2
15.230.74.72 -> 2
15.230.41.100 -> 5
15.230.74.125 -> 3
150.222.135.223 -> 9
150.222.135.40 -> 20
13.34.7.64 -> 5
15.230.74.42 -> 4


In [5]:
# count rows
print("Number of rows in data_normal: " + str(len(data_normal)))
print("Number of rows in data_attack: " + str(len(data_attack)))

# Count number of packets to port 53
data_normal_dns = data_normal[data_normal['port'] == 53]
data_attack_dns = data_attack[data_attack['port'] == 53]

# Count number of packets to port 53 by dst_ip
data_normal_dns = data_normal_dns.groupby('dst_ip')['dst_ip'].count()
data_attack_dns = data_attack_dns.groupby('dst_ip')['dst_ip'].count()

# Show
print("Number of packets to port 53 by dst_ip in data_normal:")
for ip in data_normal_dns.index:
    print("IP: " + ip + " - " + str(data_normal_dns[ip]) + " packets" + ", protocol: " + str(data_normal[data_normal['dst_ip'] == ip]['proto'].unique()))

print("\nNumber of packets to port 53 by dst_ip in data_attack:")
for ip in data_attack_dns.index:
    print("IP: " + ip + " - " + str(data_attack_dns[ip]) + " packets" + ", protocol: " + str(data_attack[data_attack['dst_ip'] == ip]['proto'].unique()))

# verify when dst_ip is 192.168.100.224 or 192.168.100.225

Number of rows in data_normal: 980027
Number of rows in data_attack: 1061078
Number of packets to port 53 by dst_ip in data_normal:
IP: 192.168.100.224 - 58048 packets, protocol: ['udp']
IP: 192.168.100.225 - 57922 packets, protocol: ['udp']

Number of packets to port 53 by dst_ip in data_attack:
IP: 192.168.100.224 - 112900 packets, protocol: ['udp']
IP: 192.168.100.225 - 115214 packets, protocol: ['udp']
